<a href="https://colab.research.google.com/github/wanasyraf4/pyspark/blob/main/pyspark_tutorial_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
from google.colab import drive
drive.mount('/content/drive/')
path = "/content/drive/My Drive/01. Data Stories vs Dashboard/pyspark data/test1.csv"
path = "/content/drive/My Drive/01. Data Stories vs Dashboard/pyspark data/test1.csv"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [28]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession

**Pyspark ML**

In [29]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Missing').getOrCreate()

In [30]:
## Read The dataset
training = spark.read.csv(path,header=True,inferSchema=True)

In [31]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [32]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [33]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [34]:
# [Age,Experience]----> new feature--->independent feature

In [35]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [36]:
output=featureassembler.transform(training)

In [37]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [38]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [39]:
finalized_data=output.select("Independent Features","Salary")

In [40]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [41]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [42]:
### Coefficients
regressor.coefficients

DenseVector([28.4757, 1271.3568])

In [43]:
### Intercepts
regressor.intercept

14299.83249581293

In [44]:
### Prediction
pred_results=regressor.evaluate(test_data)

In [45]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|         [31.0,10.0]| 30000|27896.147403685143|
+--------------------+------+------------------+



In [46]:

pred_results.meanAbsoluteError,pred_results.meanSquaredError

(2103.852596314857, 4426195.747020763)